In [ ]:
%pip install langchain

#import libraries
import openai
import langchain
import pinecone 
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


In [ ]:
from dotenv import load_dotenv
load_dotenv()




In [ ]:
import os

In [ ]:
def read_doc(directory_path):
    file_loader = PyPDFDirectoryLoader(directory_path)
    documents = file_loader.load()
    return documents

In [ ]:
doc = read_doc('document/')
len(doc)


In [ ]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return docs

In [ ]:
documents=chunk_data(docs=doc)
len(documents)

In [ ]:


embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

In [ ]:
vectors = embeddings.embed_query("how are you?")
vectors

In [ ]:
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment="gcp-starter"
)
index_name="langchainvector"

In [ ]:
index=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [ ]:
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [ ]:
llm=OpenAI(model_name="text-davinci-003",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [ ]:
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [ ]:
our_query = "How much the agriculture target will be increased by how many crore?"
answer = retrieve_answers(our_query)
print(answer)